In [2]:
import pandas as pd
from datetime import datetime

In [3]:
def get_new_weight_class_for_fighter(fighter_id, ufcFights):
    fighter_fights = ufcFights[(ufcFights['fighter1_id'] == fighter_id) | (ufcFights['fighter2_id'] == fighter_id)]
    
    fighter_fights = fighter_fights[fighter_fights['weightclass'] != 'Open Weight']
    
    if not fighter_fights.empty:
        # Find the weight class of the first fight in a normal weight class
        first_normal_weight_fight = fighter_fights.iloc[0]
        return first_normal_weight_fight['weightclass']
    
    # If no non-"Open Weight" fight is found, return "Unknown"
    return 'Unknown'

In [4]:
def calculateKValue(totalFights, title_fight, won):
    k = max(80 - 2 * totalFights, 40) * 1.0
    if title_fight == "Yes" and won == 1:
        k *= 1.5
    return k

In [5]:
def expectedValue(fighter1Rating, fighter2Rating):
    expected = 1 / (1 + pow(10, ((fighter2Rating - fighter1Rating) /400)))
    return expected

In [6]:
def updateElo(fighterTable, fightTable, current_fight_id, fighter1_id, fighter2_id, title_fight, method, outcome):
    fighter1Rating = fighterTable.loc[fighterTable['fighter_id'] == fighter1_id, 'currentElo'].values[0]
    fighter2Rating = fighterTable.loc[fighterTable['fighter_id'] == fighter2_id, 'currentElo'].values[0]
    fighter1MaxRating = fighterTable.loc[fighterTable['fighter_id'] == fighter1_id, 'maxElo'].values[0]
    fighter2MaxRating = fighterTable.loc[fighterTable['fighter_id'] == fighter2_id, 'maxElo'].values[0]

    fightsUntilDate = fightTable[fightTable['fight_id'] < current_fight_id]

    fighter1FightsNum = len(fightsUntilDate[(fightsUntilDate['fighter1_id'] == fighter1_id) |(fightsUntilDate['fighter2_id'] == fighter1_id)])
    fighter2FightsNum = len(fightsUntilDate[(fightsUntilDate['fighter1_id'] == fighter2_id) |(fightsUntilDate['fighter2_id'] == fighter2_id)])

    won = 1
    lost = 0
    if outcome !=  "win":
        won = 0

    fighter1K = calculateKValue(fighter1FightsNum, title_fight, won)
    fighter2K = calculateKValue(fighter2FightsNum, title_fight, lost)

    eventA = 1.0
    eventB = 0.0
    if outcome == "draw" or outcome == "nc":
        eventA = 0.5
        eventB = 0.5
    
    fighter1Expected = expectedValue(fighter1Rating, fighter2Rating)
    fighter2Expected = expectedValue(fighter2Rating, fighter1Rating)

    newFighter1Elo = fighter1Rating + fighter1K * (eventA - fighter1Expected)
    newFighter2Elo = fighter2Rating + fighter2K * (eventB - fighter2Expected)
    if method == "KO/TKO":
        newFighter1Elo += 10
        newFighter2Elo -= 10
    elif method == "SUB":
        newFighter1Elo += 8
        newFighter2Elo -= 8

    fighterTable.loc[fighterTable['fighter_id'] == fighter1_id, 'currentElo'] = newFighter1Elo
    fighterTable.loc[fighterTable['fighter_id'] == fighter2_id, 'currentElo'] = newFighter2Elo

    if newFighter1Elo > fighter1MaxRating:
        fighterTable.loc[fighterTable['fighter_id'] == fighter1_id, 'maxElo'] = newFighter1Elo
    if newFighter2Elo > fighter2MaxRating:
        fighterTable.loc[fighterTable['fighter_id'] == fighter2_id, 'maxElo'] = newFighter2Elo
    

    return fighterTable

Redundant Now (Creates a new Elo and removed fighters)


In [7]:
"""
ufcFights = pd.read_csv("ufcFights.csv")
ufcEvents = pd.read_csv("ufcEvent.csv")
ufcFighters = pd.read_csv("ufcFighters.csv")

ufcFights = ufcFights[ufcFights["event_id"] >= 32]
ufcFighters = ufcFighters.drop(columns= ["currentElo", "highestElo"])
ufcFighters["currentElo"] = 1500
ufcFighters["maxElo"] = 1500



for index, fight in ufcFights.iterrows():
    # Extract necessary details from the current fight row
    current_fight_id = fight['fight_id']
    fighter1_id = fight['fighter1_id']
    fighter2_id = fight['fighter2_id']
    title_fight = fight['title_fight']  # Assuming there's a column 'title_fight' indicating if it's a title fight
    method = fight['method']  # The method of victory, e.g., KO/TKO, SUB
    outcome = fight['outcome']  # Outcome of the fight: "win", "loss", "draw"
    
    # Call updateElo for this fight to update the Elo ratings
    ufcFighters = updateElo(ufcFighters, ufcFights, current_fight_id, fighter1_id, fighter2_id, title_fight, method, outcome)

# Check the updated fighter table
#ufcFighters.tail()"
fighters_to_keep = []

for index, fighter in ufcFighters.iterrows():
    fighter_id = fighter['fighter_id']
    weightclass = fighter['weightclass']
    currentElo = fighter['currentElo']

    if weightclass == 'Open Weight' and currentElo == 1500:
        fights_after_threshold = ufcFights[ufcFights['fighter1_id'] == fighter_id]
        fights_after_threshold = fights_after_threshold[ fights_after_threshold['event_id'] >= 32]
        if len(fights_after_threshold) == 0:
                continue 
        new_weight_class = get_new_weight_class_for_fighter(fighter_id, ufcFights)  # Implement this function as needed
        fighter['weightclass'] = new_weight_class
    
    fighters_to_keep.append(fighter)

updated_fighters = pd.DataFrame(fighters_to_keep)
updated_fighters.to_csv("ufcFightersNew.csv", index=False)
"""


'\nufcFights = pd.read_csv("ufcFights.csv")\nufcEvents = pd.read_csv("ufcEvent.csv")\nufcFighters = pd.read_csv("ufcFighters.csv")\n\nufcFights = ufcFights[ufcFights["event_id"] >= 32]\nufcFighters = ufcFighters.drop(columns= ["currentElo", "highestElo"])\nufcFighters["currentElo"] = 1500\nufcFighters["maxElo"] = 1500\n\n\n\nfor index, fight in ufcFights.iterrows():\n    # Extract necessary details from the current fight row\n    current_fight_id = fight[\'fight_id\']\n    fighter1_id = fight[\'fighter1_id\']\n    fighter2_id = fight[\'fighter2_id\']\n    title_fight = fight[\'title_fight\']  # Assuming there\'s a column \'title_fight\' indicating if it\'s a title fight\n    method = fight[\'method\']  # The method of victory, e.g., KO/TKO, SUB\n    outcome = fight[\'outcome\']  # Outcome of the fight: "win", "loss", "draw"\n    \n    # Call updateElo for this fight to update the Elo ratings\n    ufcFighters = updateElo(ufcFighters, ufcFights, current_fight_id, fighter1_id, fighter2_id

In [8]:
def getLastFightDate(fighter_id, ufcFights):
    fighterFights = ufcFights[(ufcFights['fighter1_id'] == fighter_id) | (ufcFights['fighter2_id'] == fighter_id)]
    
    # Get the most recent fight date
    latest_fight_date = fighterFights['date'].max()
    
    return latest_fight_date

In [9]:
def removeInactiveFighters(ufcFighters, ufcFights, years):
    currentDate = datetime.now()

    activeFighters = []

    for index, fighter in ufcFighters.iterrows():
        fighter_id = fighter['fighter_id']

        lastFightDate = getLastFightDate(fighter_id, ufcFights)

        if lastFightDate is not pd.NaT:
            if (currentDate - lastFightDate).days <= years * 365:
                activeFighters.append(fighter)
        
    activeFightersDf = pd.DataFrame(activeFighters)
    activeFightersDf.to_csv("ufcActiveFighters.csv")
    

Irrelevant Now, want to dynamically add events and keep updaying the active list later

In [10]:
"""
ufcFights = pd.read_csv("ufcFights.csv")
ufcEvents = pd.read_csv("ufcEvent.csv")
ufcFighters = pd.read_csv("ufcFightersNew.csv")
ufcEvents['date'] = pd.to_datetime(ufcEvents['date'])

ufcFights = pd.merge(ufcFights, ufcEvents[['event_id', 'date']], on='event_id')
activeFightersDf = removeInactiveFighters(ufcFighters, ufcFights, 2)


"""

'\nufcFights = pd.read_csv("ufcFights.csv")\nufcEvents = pd.read_csv("ufcEvent.csv")\nufcFighters = pd.read_csv("ufcFightersNew.csv")\nufcEvents[\'date\'] = pd.to_datetime(ufcEvents[\'date\'])\n\nufcFights = pd.merge(ufcFights, ufcEvents[[\'event_id\', \'date\']], on=\'event_id\')\nactiveFightersDf = removeInactiveFighters(ufcFighters, ufcFights, 2)\n\n\n'

In [15]:
fighters = pd.read_csv("ufcActiveFighters.csv")
top_50_elo = fighters.sort_values(by='currentElo', ascending=False).head(50)

# Display the top 10 fighters with the highest Elo
print(top_50_elo[['fighter_id', 'name', 'weightclass', 'currentElo', 'maxElo']])

     fighter_id                   name           weightclass   currentElo  \
5           596              Jon Jones     Light Heavyweight  2098.418582   
97         1438        Islam Makhachev           Lightweight  2061.930822   
350        2100      Dricus Du Plessis          Middleweight  1950.929287   
22          881           Stipe Miocic           Heavyweight  1942.726219   
101        1455           Kamaru Usman          Welterweight  1932.827040   
351        2102           Ilia Topuria         Featherweight  1928.945782   
46         1117           Amanda Nunes  Women's Bantamweight  1928.579757   
26          929           Max Holloway         Featherweight  1926.715804   
137        1612  Alexander Volkanovski           Lightweight  1926.253093   
425        2215           Alex Pereira          Middleweight  1914.230487   
76         1357           Leon Edwards          Welterweight  1908.076953   
196        1773       Magomed Ankalaev     Light Heavyweight  1906.789086   